In [ ]:
from IPython.display import clear_output
import random

In [ ]:
%run sim_network.ipynb
%run berkeley.ipynb
%run util.ipynb
%run show_plt.ipynb

In [ ]:
print_g_route()

In [ ]:
stride = float(config.get('config', 'simulate_stride'))
true_time = 0.0
error_nodes_data = {}
clocks = get_clocks(nodes)
main_clock = get_main_clock(clocks)
print(f"main clock = {main_clock.show_name}")
slave_clocks = get_slave_clocks(clocks,main_clock)
error_nodes_data['true time'] = []
for clock in clocks:
    error_nodes_data[clock] = []

def set_error_nodes_data(error_nodes_data):
    error_nodes_data['true time'].append(true_time)
    for clock in clocks:
        error_nodes_data[clock].append(clock.time)

In [ ]:
remove_file(config.get('config', 'output_clocks_delay'),"*.csv")
remove_file(config.get('config', 'output_link_info'),"*.csv")
remove_file(config.get('config', 'output_visualization'),"*.png")
step = 0
while step < int(config.get('config', 'simulate_max_steps')):
    step+=1
    print(f'\nstep{step}---------------------------------------------------------------')
    print(f'          {format_time_units()}          {format_time_units()}')
    print(f'True Time {format_time(true_time)}\n')
    set_error_nodes_data(error_nodes_data)
    for clock in clocks:
        print(f'{clock.show_name}    {format_time(clock.time)}    Error:{format_time(clock.time - true_time)}')
        clock.time += stride+clockDeltaTime(step*stride)
    true_time += stride
    reset_all_links_delay(links,package_size)
    for slave_clock in slave_clocks:
        clear_all_route(g,routes,links)
        create_best_route(main_clock.name,slave_clock.name)
        set_default_properties(links,nodes)
        print_g_route(f"output/visualization/{step}_{main_clock.show_name} To {slave_clock.show_name}.png")
    save_all_links_delay(links,step)
    if step % 10 == 0:
        new_time = run_berkeley_algorithm(main_node=clock.show_name,step=step)
        print(f"Target Time :{new_time}")
        for clock in clocks:
            print(f"Correction of {clock.show_name}: {float(new_time)-clock.time}")
            clock.time = float(new_time)
        time.sleep(float(config.get('config', 'simulate_berkeley_wait_time')))
    time.sleep(float(config.get('config', 'simulate_normal_wait_time')))

In [ ]:
show_delay_plt(step)
show_clocks_time(step)
show_clocks_delay(step)
show_error_with_time(error_nodes_data,main_clock)

In [ ]:
!jupyter nbconvert --clear-output --inplace show.ipynb